# Training Models

In [1]:
#Importing Libraries
import sys
import os
import numpy as np
import pandas as pd
import mlflow

In [2]:
#importing local scripts
#Adding scripts path
sys.path.append(os.path.abspath(os.path.join('..')))
#importing data_manipulator script
from scripts.ML_modelling_utils import *

## Importing and Fixing Data

In [3]:
#Importing the collected Data
path = 'data/AdSmartABdata.csv'
repo = 'https://github.com/DePacifier/abtest-mlops'
all_dfs = import_all_data_using_tagslist(path=path, repo=repo, tags=[
                                         'chrome-mobile', 'chrome-mobile-view', 'facebook', 'samsung-internet', 'platform-6'])


In [4]:
all_dfs['chrome-mobile'].head(5)

,auction_id,experiment,date,hour,device_make,platform_os,browser,response
0,378645d2-f8cf-43c0-b1be-e699d846e596,control,2020-07-04,6,Generic Smartphone,6,Chrome Mobile,0
1,ef334830-f200-4c1b-aea2-b01b96ca0950,exposed,2020-07-03,12,Generic Smartphone,6,Chrome Mobile,0
2,49e7690d-4825-499c-9553-756af3198a10,control,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,1
3,80b9fecc-ef23-4896-bd43-b81c074f49c3,control,2020-07-09,17,Generic Smartphone,6,Chrome Mobile,1
4,3dd09b49-9b11-45bd-ab0d-e279a8e5edc7,exposed,2020-07-03,0,Generic Smartphone,6,Chrome Mobile,0


In [5]:
# Spliting the date column to year, month and day columns and removing it
all_dfs_date_fixed = split_date_to_numbers(all_dfs, 'date')
all_dfs_date_fixed['chrome-mobile'].head(5)

,auction_id,experiment,hour,device_make,platform_os,browser,response,year,month,day
0,378645d2-f8cf-43c0-b1be-e699d846e596,control,6,Generic Smartphone,6,Chrome Mobile,0,2020,7,4
1,ef334830-f200-4c1b-aea2-b01b96ca0950,exposed,12,Generic Smartphone,6,Chrome Mobile,0,2020,7,3
2,49e7690d-4825-499c-9553-756af3198a10,control,10,Generic Smartphone,6,Chrome Mobile,1,2020,7,7
3,80b9fecc-ef23-4896-bd43-b81c074f49c3,control,17,Generic Smartphone,6,Chrome Mobile,1,2020,7,9
4,3dd09b49-9b11-45bd-ab0d-e279a8e5edc7,exposed,0,Generic Smartphone,6,Chrome Mobile,0,2020,7,3


In [6]:
#drop columns from each data, the grouping column b/c its a univalue column
#We have 5 dataframes of which 4 are grouped based on browser and 1 based on platform
for i in all_dfs_date_fixed:
    if(i != "platform-6"):
        all_dfs_date_fixed[i].drop('browser', axis=1, inplace=True)
    else:
        all_dfs_date_fixed[i].drop('platform_os', axis=1, inplace=True)

In [13]:

data_type_fixed_dfs = change_columns_to_numbers(all_dfs_date_fixed, ['experiment', 'device_make', 'browser'])
data_type_fixed_dfs['platform-6'].sample(5)


,auction_id,experiment,hour,device_make,browser,response,year,month,day
873,78334283-17e0-4221-b93b-18d65beffe9b,1,13,13,1,1,2020,7,3
452,1188cf08-c5de-4fd9-94c5-ac9067cb4a1b,0,3,13,1,1,2020,7,7
720,895e5634-5e29-4bfb-8183-5e412877d691,0,0,66,3,0,2020,7,10
804,0ae1d6a1-545c-4aa0-841e-e6213623bb66,0,8,54,3,0,2020,7,6
168,234d896d-60d3-47f8-aa4d-389e81f280ec,1,0,13,1,1,2020,7,10


In [16]:
# #Get all train, validate and test sets
chrome_mobile_dict = get_train_validate_test_sets(data_type_fixed_dfs['chrome-mobile'], predicted_column='response', remove_columns=['auction_id'])
chrome_mobile_view_dict = get_train_validate_test_sets(data_type_fixed_dfs['chrome-mobile-view'], predicted_column='response', remove_columns=['auction_id'])
facebook_dict = get_train_validate_test_sets(data_type_fixed_dfs['facebook'], predicted_column='response', remove_columns=['auction_id'])
samsung_internet_dict = get_train_validate_test_sets(data_type_fixed_dfs['samsung-internet'], predicted_column='response', remove_columns=['auction_id'])
platform_6_dict = get_train_validate_test_sets(data_type_fixed_dfs['platform-6'], predicted_column='response', remove_columns=['auction_id'])

In [17]:
platform_6_dict['train_x'].head(5)

,experiment,hour,device_make,browser,year,month,day
595,1,21,13,1,2020,7,9
143,0,7,27,2,2020,7,6
811,0,5,13,1,2020,7,6
848,1,13,66,5,2020,7,5
366,1,21,13,1,2020,7,5


# Training

## Logistic Regression Model